In [1]:
import tensorflow as tf
from tensorflow import keras
import open3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [16]:
re_model = keras.models.load_model("C:\\Users\\Srividya\\Downloads\\my_h5_model1n3.h5")

In [17]:
pcd09 = open3d.io.read_point_cloud('C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\03.ply')
X_test = np.asarray(pcd09.points)
print(X_test.shape)
bst = open("C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\03.label", "r").readlines()
listasInt = list(map(int, bst))
Y_test = np.array(listasInt)
Y_test = Y_test[1:]
print(Y_test.shape)

(1249801, 3)
(1249801,)


In [4]:
def samp(points,a):
  l, c =  np.unique(a , return_inverse= False ,return_counts = True) 
  prob_arr = np.zeros((a.shape[0]))
  ind = np.argsort(c)
  ls = l[ind]
  cs = c[ind]
  for i in range(a.shape[0]):
    j = np.where(ls == a[i])
  # csh = cs.shape[0]
    prob_arr[i] = cs[cs.shape[0]-j[0]-1]/((cs[j])*a.shape[0])

  arr_indices= np.arange(a.shape[0])
  arr_indices = np.random.choice(arr_indices, size=int(0.2*a.shape[0]), replace=False, p=prob_arr)
  rand_samp_points = points[arr_indices]
  rand_samp_labels = a[arr_indices]
  return rand_samp_points, rand_samp_labels


#rX_test,rY_test = samp( X_test, Y_test)
#rpoints,rlabels = samp(points,labels)


# _____________________________________________________________________________________________________________________

In [2]:
pcd01 = open3d.io.read_point_cloud('C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\01.ply')
points = np.asarray(pcd01.points)
bst01 = open("C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\01.label", "r").readlines()
listasInt01 = list(map(int, bst01))
labels01 = np.array(listasInt01)
labels = labels01[1:]

In [5]:
rpoints,rlabels = samp(points,labels)
ocd = open3d.geometry.PointCloud()
ocd.points = open3d.utility.Vector3dVector(rpoints)
#ocd.colors = open3d.utility.Vector3dVector(colors)
open3d.visualization.draw_geometries([ocd])

In [5]:
#visualisation
pcd01 = open3d.io.read_point_cloud('C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\01.ply')
points = np.asarray(pcd01.points)
ocd = open3d.geometry.PointCloud()
ocd.points = open3d.utility.Vector3dVector(points)
open3d.visualization.draw_geometries([ocd])

# ---------------------------------------------------------------------------------------------------------------

In [13]:
def room1samples(data, label, sample_num_point):
    """ Prepare whole room samples.

    Args:
        data: N x 6 numpy array, 012 are XYZ in meters, 345 are RGB in [0,1]
            assumes the data is shifted (min point is origin) and
            aligned (aligned with XYZ axis)
        label: N size uint8 numpy array from 0-12
        sample_num_point: int, how many points to sample in each sample
    Returns:
        sample_datas: K x sample_num_point x 9
                     numpy array of XYZRGBX'Y'Z', RGB is in [0,1]
        sample_labels: K x sample_num_point x 1 np array of uint8 labels
    """
    N = data.shape[0]  
    order = np.arange(N)
    np.random.shuffle(order) 
    data = data[order, :]
    label = label[order]

    batch_num = int(np.ceil(N / float(sample_num_point)))
    sample_datas = np.zeros((batch_num, sample_num_point, 3))
    sample_labels = np.zeros((batch_num, sample_num_point, 1))

    for i in range(batch_num):
        beg_idx = i*sample_num_point
        end_idx = min((i+1)*sample_num_point, N)
        num = end_idx - beg_idx
        sample_datas[i,0:num,:] = data[beg_idx:end_idx, :]
        sample_labels[i,0:num,0] = label[beg_idx:end_idx]
        if num < sample_num_point:
            makeup_indices = np.random.choice(N, sample_num_point - num)
            sample_datas[i,num:,:] = data[makeup_indices, :]
            sample_labels[i,num:,0] = label[makeup_indices]
    return sample_datas, sample_labels


In [14]:
X_tests, Y_tests = room1samples(rX_test, rY_test, sample_num_point = 30000)
a = X_tests.shape[0]
print(a)
print(X_tests.shape)
print(Y_tests.shape)

21
(21, 30000, 3)
(21, 30000, 1)


In [15]:
ps = re_model.predict(X_tests)  #pred = keras.utils.np_utils.probas_to_classes(model.predict(X_test))
pred = np.argmax(ps,axis=2)
#print("actual=%s, Predicted=%s" % (Y_tests[100,:25], pred[100,:25]))

ValueError: in user code:

    C:\Users\Srividya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\Srividya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Srividya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Srividya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Srividya\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Srividya\anacond

In [29]:
X_testing = X_tests.reshape(a*24400,3)
Y_testing = Y_tests.reshape(a*24400,1)
predicting = pred.reshape(a*24400,1)

In [2]:
g_classes = ["bowl", "cap", "cereal_box", "coffee_mug", "coffee_table", "office_chair", "soda_can", "sofa", "table", "background" ]
g_class2label = {cls: i+1 for i,cls in enumerate(g_classes)}
g_class2color = {'bowl':	[0,255/255,0],#green
                 'cap':	[0,0,1.0],        #blue
                 'cereal_box':	[0,1.0,1.0],#cyan/sky blue
                 'coffee_mug':        [1.0,1.0,0],#yellow
                 'coffee_table':      [1.0,0,1.0],#magenta/pink
                 'office_chair':      [100/255,100/255,255/255],#
                 'soda_can':        [200/255,200/255,100/255],
                 'sofa':       [170/255,120/255,200/255],
                 'table':       [255/255,0,0],
                 'background':        [200/255,100/255,100/255]}
#                  'bookcase':    [10,200,100],
#                  'board':       [200,200,200],
#                  'clutter':     [50,50,50]} 
g_label2color = {g_classes.index(cls)+1: g_class2color[cls] for cls in g_classes}

In [31]:
predicting = pred.flatten()
length = predicting.shape[0]
colors = np.empty([length,3])
for i in range(length):
    #for k in range(3):
        lab = predicting[i]
        colors[i] = g_label2color[lab]
print(colors.shape)

(260400, 3)


In [32]:
ocd = open3d.geometry.PointCloud()
ocd.points = open3d.utility.Vector3dVector(X_testing)
ocd.colors = open3d.utility.Vector3dVector(colors)
open3d.visualization.draw_geometries([ocd])

In [6]:
import tensorflow as tf
from tensorflow import keras
import open3d
import numpy as np

pcd01 = open3d.io.read_point_cloud('C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\03.ply')
X_testing = np.asarray(pcd01.points)
bst = open("C:\\Users\\Srividya\\Downloads\\rgbd-scenes-v2_pc\\rgbd-scenes-v2\\pc\\03.label", "r").readlines()
listasInt = list(map(int, bst))
Y_test = np.array(listasInt)
Y_testing = Y_test[1:]

length = Y_testing.shape[0]
colors = np.empty([length,3])
for i in range(length):
    #for k in range(3):
        lab = Y_testing[i]
        colors[i] = g_label2color[lab]

ocd = open3d.geometry.PointCloud()
ocd.points = open3d.utility.Vector3dVector(X_testing)
ocd.colors = open3d.utility.Vector3dVector(colors)
open3d.visualization.draw_geometries([ocd])


In [ ]:
import copy
X_testing_copy = copy.deepcopy(X_testing)

In [15]:
#l,m,n are coordinates of centroid, axis is that axis passing through centroid, 
#about which object has to be rotated at theta angle

import math
def getTransMatrix(p,q,r):
    a = np.identity(3)
    b = np.array([[p], [q], [r] ,[1]])
    c = np.concatenate((np.concatenate((a,np.zeros((1,3))),axis = 0), b),axis = 1)
    return c
def toRadian(a):
    return a*math.pi/180
def getRotMatrix(axis, degree):
    switcher = {
      'X': np.array([[ 1,                           0,   0                         ,0],
                     [ 0,  math.cos(toRadian(degree)),  -math.sin(toRadian(degree)),0],
                     [ 0,  math.sin(toRadian(degree)),   math.cos(toRadian(degree)),0],
                    [0,0,0,1]],dtype = float),
      'Y': np.array([[ math.cos(toRadian(degree)),  0,  math.sin(toRadian(degree)),0],
                     [                          0,  1,  0                         ,0],
                     [-math.sin(toRadian(degree)),  0,  math.cos(toRadian(degree)),0],
                    [0,0,0,1]],dtype = float),
      'Z': np.array([[ math.cos(toRadian(degree)),  -math.sin(toRadian(degree)),  0,0],
                     [ math.sin(toRadian(degree)),   math.cos(toRadian(degree)),  0,0],
                     [ 0,                            0,                           1,0],
                    [0,0,0,1]],dtype = float),
    }
    return switcher.get(axis)

def find_centroid(arr):
    c_wise_sum = np.sum(arr,axis = 0, keepdims = True)
    l = (c_wise_sum[0,0])/(arr.shape[0] - 1)
    m = (c_wise_sum[0,1])/(arr.shape[0] - 1)
    n = (c_wise_sum[0,2])/(arr.shape[0] - 1)
    return l,m,n
def rot_about_centroid(furniture,axis,theta):    
    obj_points = np.array([[0,0,0]])
    length = Y_testing.shape[0]
    k = g_class2label[furniture] 
    for i in range(length):
        if Y_testing[i] == k:
            obj_points = np.append(obj_points,np.array([[X_testing[i,0], X_testing[i,1], X_testing[i,2]]]),axis=0)
    
    l,m,n = find_centroid(obj_points)
    a = getRotMatrix(axis, theta)
    d = np.matmul(np.matmul(getTransMatrix(-l,-m,-n),getRotMatrix(axis,theta)),getTransMatrix(l,m,n))
    for i in range(length):
        if Y_testing[i] == k:
            e = np.matmul(d,np.array([[X_testing[i,0]],[X_testing[i,1]],[X_testing[i,2]],[1]]))
            X_testing_copy[i,0] = e[0,0]
            X_testing_copy[i,1] = e[1,0]
            X_testing_copy[i,2] = e[2,0]
    colors_copy = np.empty([length,3])
    for i in range(length):
    #for k in range(3):
        lab = Y_testing[i]
        colors_copy[i] = g_label2color[lab]
    bcd = open3d.geometry.PointCloud()
    bcd.points = open3d.utility.Vector3dVector(X_testing_copy)
    bcd.colors = open3d.utility.Vector3dVector(colors_copy)
    open3d.visualization.draw_geometries([bcd])
    

In [13]:

rot_about_centroid('coffee_table','X',45)

In [17]:
rot_about_centroid('coffee_table','Z',45)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
